In [21]:
import pymongo
import pandas as pd
from pymongo import MongoClient

# Connexion à MongoDB
cluster = pymongo.MongoClient("mongodb://localhost:27017/")
db = cluster["ProjectBigData"]

# Collection "topVideo"
collection = db['topVideo']

df = pd.read_csv('db/dailyTop200.csv')

# Convertir la colonne 'publishedAt' en format datetime avec le bon format
df['publishedAt'] = pd.to_datetime(df['publishedAt'], errors='coerce')

# Formater la colonne 'publishedAt' au format ISO 8601 avec le fuseau horaire UTC
df['publishedAt'] = df['publishedAt'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Conversion du DataFrame en dictionnaire pour l'insertion dans la base de données
data_dict = df.to_dict(orient='records')

# Insérer les données dans la collection MongoDB 'topVideo'
collection.insert_many(data_dict)

# Collection "comments"
collection = db['comments']

# Création de l'index unique sur le champ "id" dans la collection "comments"
collection.create_index([("id", pymongo.ASCENDING), ("videoId", pymongo.ASCENDING)], unique=True)

# Chargement des données depuis le fichier CSV "comments.csv"
df = pd.read_csv('db/comments.csv')

# Insérer les données dans la collection MongoDB 'comments'
data_dict = df.to_dict(orient='records')
for document in data_dict:
    try:
        collection.insert_one(document)
    except pymongo.errors.DuplicateKeyError as e:
        continue


ValueError: time data "2024-03-19T19:30:08Z" doesn't match format "%Y-%m-%d %H:%M:%S+00:00", at position 1555. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.